# Example Interlock Script
J. Ramirez 02-13-2026

**Brief: Subscribe to GeecsDevice and use ```InterlockServer``` class to create software interlock for your experiment.**

The InterlockServer class creates a tcp server to interface with Master Control. A user creates a server by setting the host IP and port to listen on in ```InterlockServer```.

For this example, we will create a simple camera threshold interlock by:
  1. Subscribing to an experiment device variable using the ```geecs_python_api```
  2. Creating a factory function to monitor the variable
  3. Call the interlock server class and setting a host IP/port to listen on
  4. Registering the factory function. and starting the server. 

We'll start by importing our necessary libraries/classes...

In [1]:
import sys # to allow import from parent directory
import time
sys.path.append("c:\\GEECS\\Developers Version\\source\\GEECS-Plugins\\GEECS-PythonAPI") # add parent directory to path to allow import of geecs_python_api
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice

from utils.geecs_interlock_server import InterlockServer # import the interlock server class


path config ..\..\..\..\user data\Configurations.INI


Next, we need to access our experiment and subscribe to the camera variable. 

In [2]:
# Set up the camera device and subscribe to the MaxCounts variable
GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Bella")
device_name = "CAM-PL1-TapeDrivePointing"
cam1 = GeecsDevice(device_name)
cam1.subscribe_var_values(['MaxCounts', 'MeanCounts'])

True

Once we have subscribed to the variables we want, we can define a helper function to do our threshold checking logic. ```camera_thresh_check()``` takes in the camera, the variables to monitor, the threshold and a bool alias to determine when the interlock flag should go off. ```camera_thresh_check()``` returns ```check()``` which actually returns a bool to the server.

In [3]:

# def camera_thresh_check(camera, variable_name: str, thresh: float, above: bool=True): # 
#     def check():
#         value = camera.get(variable_name)
#         if value is None:
#             return False
#         return (value > thresh) if above else (value < thresh)
#     return check

def camera_thresh_check(camera, variable_name: str, thresh: float, above: bool=True):
    value = camera.get(variable_name)
    if value is None:
        return False
    return (value > thresh) if above else (value < thresh)


Finally, we call ```InterlockServer``` and give a host IP and port. We pass interlocks to it with ```register_monitor()```. From there, the client will now see _safe_ or _not safe_ depending on our interlock logic.

In [4]:
server = InterlockServer(host='192.168.14.14', port=5001) # create the interlock server by 
# server.register_monitor('cam_counts', camera_thresh_check(cam1, 'MaxCounts', 120, above=True),0.5)
# server.register_monitor('cam_mean_counts', camera_thresh_check(cam1, 'MeanCounts', 50, above=True),0.5)

server.set_interlock("Camera MaxCounts Check", camera_thresh_check(cam1, 'MaxCounts', 120, above=True)) # add the camera max counts check to the interlock server with a threshold of 120
server.start()

print("Camera interlock server running...")
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    server.stop()

[Camera MaxCounts Check] Interlock ACTIVE
Interlock server started with 1 monitor(s)
Camera interlock server running...
Interlock server listening on 192.168.14.14:5001
Client connected: ('192.168.14.15', 49285)
Client connected: ('192.168.14.15', 49420)
Client disconnected: ('192.168.14.15', 49285)
Client disconnected: ('192.168.14.15', 49420)
Client connected: ('192.168.14.14', 58558)
Client disconnected: ('192.168.14.14', 58558)
Stopping interlock server...
Interlock server stopped
